# Autoregressive Distributed Lag Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tabulate
import statsmodels.api as sm
from statsmodels.tsa.api import ARDL
from statsmodels.tsa.ar_model import AutoReg

In [2]:
data0=pd.read_excel("ARDLdata.xlsx")

##### Data is obtained from Türkiye Central Bank for Türkiye. The time period where  from 2013-01 to 2023-02  is covered by data. All data set are index type. The question of analysis is that whether there is long run relationship between real effective exchance index and the export of consumption goods quatity index.

In [3]:
data = data0.copy()
data['year']= pd.to_datetime(data['year'], dayfirst=True)
data.set_index('year', inplace=True)
data.asfreq('M')
data.index = data.index.to_period('M')
print('rows:',data.shape[0])
print('columns:',data.shape[1])
print(data)

rows: 122
columns: 13
              excap      uexcap        exig       uexig      excons  \
year                                                                  
2013-01   72.774922  114.548865   89.549035  119.858315   78.879185   
2013-02   87.779822  115.929133   93.659720  120.523347   83.823380   
2013-03   89.235707  113.086478  104.075595  119.110688   90.611344   
2013-04   83.719653  113.187796  100.123254  117.845872   88.513588   
2013-05   90.333201  114.163612  105.533807  116.702413   92.782109   
...             ...         ...         ...         ...         ...   
2022-10  175.110803  113.174784  142.459137  123.157421  162.892306   
2022-11  174.445293  117.248701  146.027841  124.733818  161.232846   
2022-12  185.873058  121.737568  146.362538  127.002810  168.916898   
2023-01  134.376116  124.815086  129.953531  126.489730  136.593404   
2023-02  138.785520  126.259345  123.623595  127.260331  126.886816   

            uexcons       imcap      uimcap        imi

## Describing Data

In [4]:
imcap=data.iloc[:,[6]] #---------> imported capital goods quantity index
imig=data.iloc[:,[8]] #---------> imported intermediate goods quantity index
imcons=data.iloc[:,[10]] #---------> imported consumption goods quantity index
excap=data.iloc[:,[0]] #---------> exported capital goods quantity index
exig=data.iloc[:,[2]] #---------> exported intermediate goods quantity index
excons=data.iloc[:,[4]] #---------> exported consumption goods quantity index
rexc=data.iloc[:,[12]] #---------> real effective exchange index

## Select The Optimal Lag

To start the analysis, firstly the optimal lag point have to be selected. Akaike and Schwarz information criterion are common tools for calculate the optimal lag point. The formulas had been showed below.<br><br>
LLH ------------> likelihood value of regression.<br>
k ------------> number of independent variables.<br>
n ------------> number of observation.

In [5]:
# This function takes the likelihood values.
def to_llf(p,q):                            
    model=ARDL(excons,p,rexc,q).fit()
    return model.llf

## $AIC=\frac{-2LLH}{n}+\frac{2k}{n}$ 

## $BIC=\frac{-2LLH}{n}+\frac{k*log(n)}{n}$

In [6]:
result_aic=[]
i_arr=[]
j_arr=[]
k_arr=[]
obs_arr=[]
'''In this inline the essential variables are created for calculate information criterion values.
   With this itteration, (to_llf) function is used in all possible ARDL model.
   Taking likelihood values are added to empty list object.
'''
for i in range(0,13,1):
    for j in range(1,13,1):
        k=i+j+2 #----------------------------------> number of independent variables
        obs=len(data)-(j)
        '''To take lag causes to lose value of data and to properly
            calculate AIC and SCI, the number of observations
            have to be adjusted according to how much values are losed.
            'obs' is to adjust number of observation according to max lag number.'''                    
        output_aic=to_llf(j,i)
        result_aic.append(output_aic)
        i_arr.append(i)
        j_arr.append(j)
        k_arr.append(k)
        obs_arr.append(obs)

In [7]:
i=pd.DataFrame(i_arr)
j=pd.DataFrame(j_arr)
lags_op=j.astype(str)+"-"+i.astype(str)
lags=pd.DataFrame(lags_op)

In [8]:
k=pd.DataFrame(k_arr)

In [9]:
obs=pd.DataFrame(obs_arr)

In [10]:
llf_df=pd.DataFrame(result_aic)

In [11]:
log_obs=obs.apply(np.log)
log_obs1=pd.DataFrame(log_obs)

In [12]:
AIC=(-2*llf_df+2*k)/obs  #-----------------> Akaike information criterion formula

In [13]:
BIC=(-2*llf_df+k*log_obs1)/obs #-----------------> Schwarz information criterion formula

In [14]:
AIC_df=pd.DataFrame(AIC)

In [15]:
BIC_df=pd.DataFrame(BIC)

In [16]:
info=pd.concat([lags, k, obs, llf_df, log_obs1,AIC_df,BIC_df], axis=1,ignore_index=True)

In [17]:
info

,0,1,2,3,4,5,6
0,1-0,3,121,-496.374194,4.795791,8.254119,8.323436
1,2-0,4,120,-492.646472,4.787492,8.277441,8.370358
2,3-0,5,119,-487.767217,4.779123,8.281802,8.398572
3,4-0,6,118,-483.763956,4.770685,8.301084,8.441966
4,5-0,7,117,-479.879664,4.762174,8.322729,8.487988
...,...,...,...,...,...,...,...
151,8-12,22,114,-461.089914,4.736198,8.475262,9.003300
152,9-12,23,113,-457.543057,4.727388,8.505187,9.060319
153,10-12,24,112,-453.717848,4.718499,8.530676,9.113211
154,11-12,25,111,-449.344721,4.709530,8.546752,9.157006


In [18]:
AIC_ten = info.iloc[:,5].nsmallest(10).iloc[0:-1]
BIC_ten = info.iloc[:,6].nsmallest(10).iloc[0:-1]

In [19]:
print(AIC_ten) #-------> first 10 minimum values of Akaike
print(BIC_ten)  #-------> first 10 minimum values of Schwarz

0     8.254119
24    8.261693
12    8.266717
1     8.277441
36    8.277929
2     8.281802
26    8.285785
25    8.286985
11    8.288312
Name: 5, dtype: float64
0     8.323436
12    8.359139
1     8.370358
24    8.377222
2     8.398572
13    8.406624
36    8.416564
25    8.426360
14    8.431086
Name: 6, dtype: float64


In [20]:
info.iloc[0:1,:]

,0,1,2,3,4,5,6
0,1-0,3,121,-496.374194,4.795791,8.254119,8.323436


To select optimal lag point, the all possible ARDL model had been tested and the likelihood value of them had been taked for calculate Akaike and Schwarz values. The maximum lag point is 12. Because of that, 156 diffirent model was created. To see which model has the minimum information criterion value, results had been ordered from smallest to largest. As seen above, ARDL(1,0) lag point was chosed with 8,25 AIC and 8,32 SCI.

## ARDL

After select the optimal lag point, the long run regression can be runned. In economics the long run refres the time period bigger 1 or 2 years. But in econometrics the long run refers the data which hadn't been taken the difference. The process of take difference of data causing the lose past time values causes to data has only itself time information. Because of that the long run regression refers that  the variables in this regression hadn't been taken difference.

### Long Run Regression

$ y_{t}=\alpha + \sum^m_{i=1}\beta_{1}y_{t-i} + \sum^n_{j=0}\beta_{2}x_{1t-j} + \epsilon_{t}$

In [21]:
modelARDL=ARDL(excons,1,rexc).fit()   #---------------> ARDL(1,0)

In [22]:
modelARDL.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,133.6779,18.311,7.300,0.000,97.417,169.939
excons.L1,0.3265,0.086,3.777,0.000,0.155,0.498
rexc.L0,-0.6432,0.111,-5.803,0.000,-0.863,-0.424


To confidence results of regression, some diagnostic tests should be made.

In [23]:
resids=pd.DataFrame(modelARDL.resid,columns=["resids"])

In [24]:
resids

,resids
year,
2013-02,-3.232592
2013-03,2.024526
2013-04,-1.665849
2013-05,2.271416
2013-06,-5.743521
...,...
2022-10,11.535072
2022-11,10.242098
2022-12,17.863419


### Diagnostics Tests (Autocorrelation and Heteroskedasticity Analysis)

In [25]:
var=pd.concat([resids,rexc,excons],axis=1).dropna()

### Breusch-Godfrey Test

$\epsilon_{t}=\alpha_{0}+\alpha_{j}X_{j}+\sum p_{i}\epsilon_{t-i}+u_{t} ;   i,j=1,......,k    $

In [26]:
BG=ARDL(var['resids'],1,var[['rexc','excons']],{'rexc': [0], 'excons': [1]},missing='raise',hold_back=1).fit()

In [27]:
BG.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,-28.1086,49.014,-0.573,0.567,-125.188,68.971
resids.L1,-0.1624,0.262,-0.621,0.536,-0.680,0.356
rexc.L0,0.1374,0.246,0.558,0.578,-0.351,0.625
excons.L1,0.1411,0.245,0.575,0.566,-0.345,0.627


In [29]:
print(BG.f_test('resids.L1=0'))

<F test: F=0.38534636943379924, p=0.5359735381001975, df_denom=116, df_num=1>


Durbin-h test is unsufficient to analysis of autoregressive models. Because of that, to analysis whether there is autocorrelation, Breusch-Godfrey Test had been used. Breusch-Godfrey Test is executed in three steps. Firstly, the model which testing aspect of autocorrelation is runned and obtained residuals. Then, with residuals are taken as dependent variable  a diffirent regression is runned  and first regression's independent variables and lagged values of residuals  are added as independent variables. Lastly,  the partial F test is executed on coefficent of lagged residiuals. According to results, the null hypothesis that there is not autocorrelation can not be rejected.

### ARCH LM Test

$\epsilon^2_{t}=\alpha_{0}+\sum^q_{i=1} \alpha_{i}\epsilon^2_{t-i} +u_{t}$

In [30]:
resids_sq=modelARDL.resid**2

In [31]:
ARCHlm=AutoReg(resids_sq, lags=1, trend='c').fit()

In [32]:
ARCHlm.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,190.7186,42.258,4.513,0.000,107.895,273.542
y.L1,0.1172,0.091,1.294,0.196,-0.060,0.295


In [33]:
print(ARCHlm.f_test("y.L1 = 0"))

<F test: F=1.6747296137323235, p=0.19815329523272995, df_denom=118, df_num=1>


To analysis heteroskedasticity, ARCH LM test had been used. The square of residuals refers to variance of regression. If $\epsilon^2_{t}$ has a relationship with lagged values of itself, the hypothesis of test that there is not heteroskedasticity $\alpha_{i}$ can be rejected. According to partial F test result, null hypothesis can nat be rejected. So, $\alpha_{i}=0$.

### Bound Test

$ \Delta{y_{t}}=\alpha + \sum^m_{i=1}\beta_{1}\Delta{y_{t-i}} + \sum^m_{i=1}\beta_{2}\Delta{x_{1t-i}} + \beta_{3}y_{t-1} + \beta_{4}x_{t-1} + \epsilon_{t}$

In [34]:
dexcons=excons.diff().fillna(0)
dexcons=np.array(dexcons)

In [35]:
drexc=np.array(rexc.diff().fillna(0))

In [36]:
rexc=np.array(rexc)

In [37]:
excons=np.array(excons)

In [38]:
exog=np.column_stack((rexc,drexc,excons))

In [39]:
bound_test=ARDL(dexcons,1,exog,{0:[1],1:[1],2:[1]}).fit()

In [40]:
bound_test.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,134.6199,22.502,5.983,0.000,90.052,179.188
y.L1,-0.0275,0.092,-0.299,0.766,-0.210,0.155
x0.L1,-0.6467,0.128,-5.043,0.000,-0.901,-0.393
x1.L1,0.6858,0.526,1.304,0.195,-0.356,1.727
x2.L1,-0.6737,0.108,-6.262,0.000,-0.887,-0.461


In [41]:
print(bound_test.f_test(" x0.L1=x2.L1= 0"))

<F test: F=19.644355763070564, p=4.492105607093129e-08, df_denom=116, df_num=2>


Unlike previous cointegration tests, ARDL developed new method. Thanks to this method, the previous methods requirement that the series be difference stationary has been eliminated. To test the long run relationship, calculated F value compare with F table values which is created by Pesaran, Shın and Smıth. If calculated F value pass the I(1) bound, the null hypothesis can be rejected, otherwise if calculated F values keep the left side of I(0) bound, null hypothesis can not be rejected. The between I(0) and I(1) bounds is indecision zone.

Critical Value Bounds	<br>	
		
Significance----I0 Bound----I1 Bound <br>
		

5%----------------3.62----------------4.16 <br>




Calculated F values is 19.64 which is bigger than I(1) bound. <br>
$F_{calculated}(19,64) > F_{table}(4,16)$ <br> $H_{0}=There is no long run relationships.$ ----> can be rejected.

### Cointegrating Form

$\Delta{y_{t}}=\alpha +  \sum^m_{i=1}\beta_{2}\Delta{x_{t}}+ECT+e_{t-1}$

##### Error Correction Term

$Y_{t}=X_{t}\beta+\epsilon_{t}$ <br><br>
$Y_{t}-X_{t}\beta=\epsilon_{t}$ <br><br>
$Error Correction Term ; \epsilon_{t-1}$


Error correction term is residual of long run regression and used in cointegration analysis for see whether short run shock will return to balance.

In [42]:
ect=np.nan_to_num(np.array(resids.shift().fillna(0)))
drexc_=np.delete(drexc,0)
dexcons_=np.delete(dexcons,0)

In [43]:
coint_exog=np.column_stack((drexc_,ect))

In [44]:
coint=ARDL(dexcons_,0,coint_exog,{0:[0], 1:[0]},trend='n').fit()

In [45]:
coint.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
x0.L0,-0.2449,0.517,-0.473,0.637,-1.269,0.780
x1.L0,-0.6936,0.094,-7.403,0.000,-0.879,-0.508


x0.L0 is difference of the export of consumption goods index. <br>
x1.L0 is level of error correction term. <br>
<br>
Conitegrating form of model is represent to short run regression. If interpret the coefficient, it can be said that real effective exchange index is ineffective on export of consumption goods index in short run as statisticaly. Aspect of error correction term, it can be seen that coefficient has negative value and is significant as statisticaly. This show that the short run shocks will be disappeared by long run impact.

### Normalized Long Run Form

In [46]:
constant_=bound_test.params[0]
rexc_1=bound_test.params[2]
excons_1=bound_test.params[4]

In [47]:
normal_rexc=rexc_1/excons_1
normal_cons=constant_/excons_1

In [48]:
normal_rexc

0.9598806618928415

In [49]:
normal_cons

-199.81713675379766

$ y_{t}=199,81 - 0,9598X_{t} + \epsilon_{t} $

Lastly, the long run form of model is presented with normalized coefficients. According to equation in long run period, the relationship between real excange rate and export of consumption goods index have almost unit flexible connection. It show that the changes on real exchange rate has important impact on export of consumption goods index.